<a href="https://colab.research.google.com/github/Hossain-MD-Rafat/support_ticket_categorization/blob/main/Support_ticket_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><p float="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/4_RGB_McCombs_School_Brand_Branded.png" width="300" height="100"/>
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<h1><center><font size=10>Artificial Intelligence and Machine Learning</center></font></h1>
<h1><center>Introduction to Natural Language Processing - Project Debrief</center></h1>

<center><p float="center">
  <img src="https://cdn.pixabay.com/photo/2017/07/24/04/23/technical-support-2533526_1280.png" width="480"/>
</p></center>

<center><font size=5>Support Ticket Categorization</center></font>

## Note: This is a sample solution for the project. Projects will NOT be graded on the basis of how well the submission matches this sample solution. Projects will be graded on the basis of the rubric only.

## Problem Statement

### Business Context

In today's dynamic business landscape, organizations are increasingly recognizing the pivotal role customer feedback plays in shaping the trajectory of their products and services. The ability to swiftly and effectively respond to customer input not only fosters enhanced customer experiences but also serves as a catalyst for growth, prolonged customer engagement, and the nurturing of lifetime value relationships. As a dedicated Product Manager or Product Analyst, staying attuned to the voice of your customers is not just a best practice; it's a strategic imperative.

While your organization may be inundated with a wealth of customer-generated feedback and support tickets, your role entails much more than just processing these inputs. To make your efforts in managing customer experience and expectations truly impactful, you need a structured approach – a method that allows you to discern the most pressing issues, set priorities, and allocate resources judiciously. One of the most effective strategies at your disposal is to harness the power of Support Ticket Categorization.


### Objective

Develop an advanced support ticket categorization system that accurately classifies incoming tickets, assigns relevant tags based on their content, implements mechanisms and generate the first response based on the sentiment for prioritizing tickets for prompt resolution.


## Installing and Importing Necessary Libraries and Dependencies

In [ ]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.45 --force-reinstall --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.45 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 225.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 275.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 246.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
# For downloading the models from HF Hub
!pip install huggingface_hub==0.20.3 pandas==1.5.3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.
transformers 4.42.4 requires huggingface-hub<1.0,>=0.23.2, but you have huggingface-hub 0.20.3 which is incompatible.


In [ ]:
# Function to download the model from the Hugging Face model hub
from huggingface_hub import hf_hub_download

# Importing the Llama class from the llama_cpp module
from llama_cpp import Llama

# Importing the json module
import json

# for loading and manipulating data
import pandas as pd

# for time computations
import time

## Loading the Data

In [ ]:
data = pd.read_csv("/content/Support_ticket_text_data_mid_term (2).csv")

## Data Overview

### Checking the first 5 rows of the data

In [ ]:
data.head()

,support_tick_id,support_ticket_text
0,ST2023-006,My internet connection has significantly slowe...
1,ST2023-007,Urgent help required! My laptop refuses to sta...
2,ST2023-008,I've accidentally deleted essential work docum...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...
4,ST2023-010,"My smartphone battery is draining rapidly, eve..."


### Checking the shape of the data

In [ ]:
data.shape

(21, 2)

* There are 21 rows and 2 columns

### Checking the missing values in the data

In [ ]:
data.isnull().sum()

support_tick_id        0
support_ticket_text    0
dtype: int64

* There are no missing values in the data

## Model Building

### Loading the model

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

In [ ]:
#uncomment the below snippet of code if the runtime is connected to GPU.
llm = Llama(
    model_path=model_path,
    n_ctx=1024,
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loade

In [ ]:
#uncomment the below snippet of code if the runtime is connected to CPU only.
#llm = Llama(
#    model_path=model_path,
#    n_ctx=1024,
#    n_cores=-2
#)

### Utility function

In [ ]:
# defining a function to parse the JSON output from the model
def extract_json_data(json_str):
    try:
        # Find the indices of the opening and closing curly braces
        json_start = json_str.find('{')
        json_end = json_str.rfind('}')

        if json_start != -1 and json_end != -1:
            extracted_category = json_str[json_start:json_end + 1]  # Extract the JSON object
            data_dict = json.loads(extracted_category)
            return data_dict
        else:
            print(f"Warning: JSON object not found in response: {json_str}")
            return {}
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}

## Task 1: Ticket Categorization and Returning Structured Output

In [ ]:
#Defining the response funciton for Task 1.
def response_1(prompt,ticket):
    start = time.time()
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      A:
      """,
      max_tokens=32,
      stop=["Q:", "\n"],
      temperature=0.01,
      echo=False,
    )
    end = time.time()

    temp_output = model_output["choices"][0]["text"]
    final_output = temp_output[temp_output.index('{'):]

    return final_output

In [ ]:
# creating a copy of the data
data_1 = data.copy()

In [ ]:
prompt_1 = """
    You are a technical assistant. Classify the support ticket based on the Support Ticket Text presented
    in the input into the following categories and not any other.
      - Technical issues
      - Hardware issues
      - Data recovery
    Return only a structured JSON output in the following format:
    {"Category": "category_prediction"}
"""

In [ ]:
start = time.time()
data_1['model_response'] = data_1['support_ticket_text'].apply(lambda x: response_1(prompt_1, x))
end = time.time()


llama_print_timings:        load time =    2509.16 ms
llama_print_timings:      sample time =       5.49 ms /    10 runs   (    0.55 ms per token,  1822.49 tokens per second)
llama_print_timings: prompt eval time =    2508.87 ms /   141 tokens (   17.79 ms per token,    56.20 tokens per second)
llama_print_timings:        eval time =    7548.51 ms /     9 runs   (  838.72 ms per token,     1.19 tokens per second)
llama_print_timings:       total time =   10100.53 ms /   150 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    2509.16 ms
llama_print_timings:      sample time =       4.84 ms /     9 runs   (    0.54 ms per token,  1860.27 tokens per second)
llama_print_timings: prompt eval time =    1687.32 ms /    51 tokens (   33.08 ms per token,    30.23 tokens per second)
llama_print_timings:        eval time =    6792.85 ms /     8 runs   (  849.11 ms per token,     1.18 tokens per second)
llama_print_timings:       total time =    8524.19 ms /    59 

In [ ]:
print("Time taken ",(end-start))

Time taken  275.47234439849854


In [ ]:
data_1['model_response']

0     {"Category": "Technical issues"}
1      {"Category": "Hardware issues"}
2        {"Category": "Data recovery"}
3     {"Category": "Technical issues"}
4     {"Category": "Technical issues"}
5     {"Category": "Technical issues"}
6     {"Category": "Technical issues"}
7      {"Category": "Hardware issues"}
8        {"Category": "Data recovery"}
9      {"Category": "Hardware issues"}
10       {"Category": "Data recovery"}
11     {"Category": "Hardware issues"}
12       {"Category": "Data recovery"}
13       {"Category": "Data recovery"}
14     {"Category": "Hardware issues"}
15    {"Category": "Technical issues"}
16    {"Category": "Technical issues"}
17       {"Category": "Data recovery"}
18       {"Category": "Data recovery"}
19    {"Category": "Technical issues"}
20    {"Category": "Technical issues"}
Name: model_response, dtype: object

In [ ]:
i = 2
print(data_1.loc[i, 'support_ticket_text'])

I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?


In [ ]:
print(data_1.loc[i, 'model_response'])

{"Category": "Data recovery"}


In [ ]:
# applying the function to the model response
data_1['model_response_parsed'] = data_1['model_response'].apply(extract_json_data)
data_1['model_response_parsed'].head()

0    {'Category': 'Technical issues'}
1     {'Category': 'Hardware issues'}
2       {'Category': 'Data recovery'}
3    {'Category': 'Technical issues'}
4    {'Category': 'Technical issues'}
Name: model_response_parsed, dtype: object

In [ ]:
data_1['model_response_parsed'].value_counts()

{'Category': 'Technical issues'}    9
{'Category': 'Data recovery'}       7
{'Category': 'Hardware issues'}     5
Name: model_response_parsed, dtype: int64

In [ ]:
# Normalizing the model_response_parsed column
model_response_parsed_df_1 = pd.json_normalize(data_1['model_response_parsed'])
model_response_parsed_df_1.head()

,Category
0,Technical issues
1,Hardware issues
2,Data recovery
3,Technical issues
4,Technical issues


In [ ]:
# Concatinating two dataframes
data_with_parsed_model_output_1 = pd.concat([data_1, model_response_parsed_df_1], axis=1)
data_with_parsed_model_output_1.head()

,support_tick_id,support_ticket_text,model_response,model_response_parsed,Category
0,ST2023-006,My internet connection has significantly slowe...,"{""Category"": ""Technical issues""}",{'Category': 'Technical issues'},Technical issues
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{""Category"": ""Hardware issues""}",{'Category': 'Hardware issues'},Hardware issues
2,ST2023-008,I've accidentally deleted essential work docum...,"{""Category"": ""Data recovery""}",{'Category': 'Data recovery'},Data recovery
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{""Category"": ""Technical issues""}",{'Category': 'Technical issues'},Technical issues
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","{""Category"": ""Technical issues""}",{'Category': 'Technical issues'},Technical issues


In [ ]:
# Dropping model_response and model_response_parsed columns
final_data_1 = data_with_parsed_model_output_1.drop(['model_response','model_response_parsed'], axis=1)
final_data_1.head()

,support_tick_id,support_ticket_text,Category
0,ST2023-006,My internet connection has significantly slowe...,Technical issues
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware issues
2,ST2023-008,I've accidentally deleted essential work docum...,Data recovery
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Technical issues
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Technical issues


## Task 2: Creating Tags

In [ ]:
# creating a copy of the data
data_2 = data.copy()

In [ ]:
def response_2(prompt,ticket,category):
    start = time.time()
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      Category: {category}
      A:
      """,
      max_tokens=64,
      stop=["Q:", "\n"],
      temperature=0.01,
      echo=False,
    )
    end = time.time()

    temp_output = model_output["choices"][0]["text"]
    final_output = temp_output[temp_output.index('{'):]

    return final_output

In [ ]:
prompt_2 = """
    You are a technical assistant. You are provided a support ticket and category as inputs. Based on the text in the ticket and category provided for the ticket,
    identify the following:
      - Tags for the ticket.

    Instructions:
    1. Carefully read the given support ticket.
    2. Analyse the given support ticket and identify the important keywords which describes the issue in the ticket.
    3. Tags can be a list of keywords too.
    4. Make sure it follows the python structure.
    5. Number of tags can be atmost 3.

    Return only a structured JSON output in the following format:
    {"Tags": "tags_prediction"}
"""

In [ ]:
start = time.time()
data_2["model_response"]=final_data_1[['support_ticket_text','Category']].apply(lambda x: response_2(prompt_2, x[0],x[1]),axis =1)
end = time.time()

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2509.16 ms
llama_print_timings:      sample time =      10.09 ms /    20 runs   (    0.50 ms per token,  1983.14 tokens per second)
llama_print_timings: prompt eval time =    2460.72 ms /   212 tokens (   11.61 ms per token,    86.15 tokens per second)
llama_print_timings:        eval time =   16084.16 ms /    19 runs   (  846.53 ms per token,     1.18 tokens per second)
llama_print_timings:       total time =   18626.04 ms /   231 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    2509.16 ms
llama_print_timings:      sample time =      12.66 ms /    24 runs   (    0.53 ms per token,  1896.33 tokens per second)
llama_print_timings: prompt eval time =    1851.00 ms /    58 tokens (   31.91 ms per token,    31.33 tokens per second)
llama_print_timings:        eval time =   19522.25 ms /    23 runs   (  848.79 ms per token,     1.18 tokens per second)
llama_print_timings:       to

In [ ]:
print("Time taken ",end-start)

Time taken  383.2012782096863


In [ ]:
data_2['model_response'].values

array(['{"Tags": ["Internet connection", "Slow internet", "Disconnections"]}',
       '{"Tags": ["Hardware issue", "Laptop start-up", "Critical presentation"]}',
       '{"Tags": ["data loss", "data recovery"]}',
       '{"Tags": ["Wi-Fi", "Signal", "Weak", "Troubleshooting"]}',
       '{"Tags": ["Battery", "Draining", "Smartphone"]}',
       '{"Tags": ["Password reset", "Online banking", "Account access"]}',
       '{"Tags": ["performance", "optimization"]}',
       '{"Tags": ["Blue Screen", "Hardware", "Crash"]}',
       '{"Tags": ["External Hard Drive", "Data Recovery"]}',
       '{"Tags": ["Graphics Card", "Hardware", "Performance"]}',
       '{"Tags": ["accidental_format", "data_recovery"]}',
       '{"Tags": ["screen_issue", "hardware"]}',
       '{"Tags": ["Water Damage", "Data Recovery"]}',
       '{"Tags": ["Physical Damage", "Data Recovery"]}',
       '{"Tags": ["touchpad", "hardware"]}',
       '{"Tags": ["Internet connection", "Frequent drops", "Work"]}',
       '{"Tags": [

In [ ]:
i = 2
print(data_2.loc[i, 'support_ticket_text'])

I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?


In [ ]:
print(data_2.loc[i, 'model_response'])

{"Tags": ["data loss", "data recovery"]}


In [ ]:
# Applying the function to the model response
data_2['model_response_parsed'] = data_2['model_response'].apply(extract_json_data)

In [ ]:
data_2["model_response_parsed"]

0     {'Tags': ['Internet connection', 'Slow interne...
1     {'Tags': ['Hardware issue', 'Laptop start-up',...
2              {'Tags': ['data loss', 'data recovery']}
3     {'Tags': ['Wi-Fi', 'Signal', 'Weak', 'Troubles...
4       {'Tags': ['Battery', 'Draining', 'Smartphone']}
5     {'Tags': ['Password reset', 'Online banking', ...
6             {'Tags': ['performance', 'optimization']}
7        {'Tags': ['Blue Screen', 'Hardware', 'Crash']}
8     {'Tags': ['External Hard Drive', 'Data Recover...
9     {'Tags': ['Graphics Card', 'Hardware', 'Perfor...
10     {'Tags': ['accidental_format', 'data_recovery']}
11               {'Tags': ['screen_issue', 'hardware']}
12          {'Tags': ['Water Damage', 'Data Recovery']}
13       {'Tags': ['Physical Damage', 'Data Recovery']}
14                   {'Tags': ['touchpad', 'hardware']}
15    {'Tags': ['Internet connection', 'Frequent dro...
16         {'Tags': ['WiFi', 'Connectivity', 'Router']}
17     {'Tags': ['accidental_format', 'data_reco

In [ ]:
# Normalizing the model_response_parsed column
model_response_parsed_df_2 = pd.json_normalize(data_2['model_response_parsed'])
model_response_parsed_df_2.head()

,Tags
0,"[Internet connection, Slow internet, Disconnec..."
1,"[Hardware issue, Laptop start-up, Critical pre..."
2,"[data loss, data recovery]"
3,"[Wi-Fi, Signal, Weak, Troubleshooting]"
4,"[Battery, Draining, Smartphone]"


In [ ]:
# Concatinating two dataframes
data_with_parsed_model_output_2 = pd.concat([data_2, model_response_parsed_df_2], axis=1)
data_with_parsed_model_output_2.head()

,support_tick_id,support_ticket_text,model_response,model_response_parsed,Tags
0,ST2023-006,My internet connection has significantly slowe...,"{""Tags"": [""Internet connection"", ""Slow interne...","{'Tags': ['Internet connection', 'Slow interne...","[Internet connection, Slow internet, Disconnec..."
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{""Tags"": [""Hardware issue"", ""Laptop start-up"",...","{'Tags': ['Hardware issue', 'Laptop start-up',...","[Hardware issue, Laptop start-up, Critical pre..."
2,ST2023-008,I've accidentally deleted essential work docum...,"{""Tags"": [""data loss"", ""data recovery""]}","{'Tags': ['data loss', 'data recovery']}","[data loss, data recovery]"
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{""Tags"": [""Wi-Fi"", ""Signal"", ""Weak"", ""Troubles...","{'Tags': ['Wi-Fi', 'Signal', 'Weak', 'Troubles...","[Wi-Fi, Signal, Weak, Troubleshooting]"
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","{""Tags"": [""Battery"", ""Draining"", ""Smartphone""]}","{'Tags': ['Battery', 'Draining', 'Smartphone']}","[Battery, Draining, Smartphone]"


In [ ]:
# Dropping model_response and model_response_parsed columns
final_data_2 = data_with_parsed_model_output_2.drop(['model_response','model_response_parsed'], axis=1)
final_data_2.head()

,support_tick_id,support_ticket_text,Tags
0,ST2023-006,My internet connection has significantly slowe...,"[Internet connection, Slow internet, Disconnec..."
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"[Hardware issue, Laptop start-up, Critical pre..."
2,ST2023-008,I've accidentally deleted essential work docum...,"[data loss, data recovery]"
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"[Wi-Fi, Signal, Weak, Troubleshooting]"
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","[Battery, Draining, Smartphone]"


In [ ]:
# Checking the value counts of Category column
final_data_2['Tags'].value_counts()

[accidental_format, data_recovery]                                                        2
[External Hard Drive, Data Recovery]                                                      2
[Internet connection, Slow internet, Disconnections]                                      1
[Internet connection, Slow internet speed, Frequent disconnections, Productivity loss]    1
[WiFi, Connectivity, Router]                                                              1
[Internet connection, Frequent drops, Work]                                               1
[touchpad, hardware]                                                                      1
[Physical Damage, Data Recovery]                                                          1
[Water Damage, Data Recovery]                                                             1
[screen_issue, hardware]                                                                  1
[Graphics Card, Hardware, Performance]                                          

In [ ]:
final_data_2 = pd.concat([final_data_2,final_data_1["Category"]],axis=1)

In [ ]:
final_data_2 = final_data_2[["support_tick_id","support_ticket_text","Category","Tags"]]
final_data_2

,support_tick_id,support_ticket_text,Category,Tags
0,ST2023-006,My internet connection has significantly slowe...,Technical issues,"[Internet connection, Slow internet, Disconnec..."
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware issues,"[Hardware issue, Laptop start-up, Critical pre..."
2,ST2023-008,I've accidentally deleted essential work docum...,Data recovery,"[data loss, data recovery]"
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Technical issues,"[Wi-Fi, Signal, Weak, Troubleshooting]"
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Technical issues,"[Battery, Draining, Smartphone]"
5,ST2023-011,I'm locked out of my online banking account an...,Technical issues,"[Password reset, Online banking, Account access]"
6,ST2023-012,"My computer's performance is sluggish, severel...",Technical issues,"[performance, optimization]"
7,ST2023-013,I'm experiencing a recurring blue screen error...,Hardware issues,"[Blue Screen, Hardware, Crash]"
8,ST2023-014,My external hard drive isn't being recognized ...,Data recovery,"[External Hard Drive, Data Recovery]"
9,ST2023-015,The graphics card in my gaming laptop seems to...,Hardware issues,"[Graphics Card, Hardware, Performance]"


## Task 3: Predicting Priority and ETA

In [ ]:
# creating a copy of the data
data_3 = data.copy()

In [ ]:
def response_3(prompt,ticket,category,tags):
    start = time.time()
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      Category: {category}
      Tags: {tags}
      A:
      """,
      max_tokens=64,
      stop=["Q:", "\n"],
      temperature=0.01,
      echo=False,
    )
    end = time.time()

    temp_output = model_output["choices"][0]["text"]
    final_output = temp_output[temp_output.index('{'):]

    return final_output

In [ ]:
prompt_3 = """
    You are a technical assistant. You are provided a support ticket,category and tags as input. Based on the text in the ticket,category and tags provided for the ticket,
    identify the following:
      - Priority of resolving the ticket
      - ETA for resolving the ticket

    Instructions:
    1. Carefully read the text of the support ticket and consider the overall sentiment of the review while assigning the priority
    2. The priority should be one of Low, Medium, or High only and not any other.
    3. All tickets cannot be of high priority.
    4. Depending on the sentiment of the review, the priority of the ticket, and the nature of the issue being faced by the customer, suggest an ETA for resolution.
    5. ETA can be one of Within 24 hours,Within 2-4 hours,Within 48 hours,ASAP only and not any other.

    Return only a structured JSON output in the following format:
    {"Priority": "priority_prediction", "ETA": "eta_prediction"}
"""

In [ ]:
# Applying generate_llama_response function on support_ticket_text column
start = time.time()
data_3['model_response'] = final_data_2[['support_ticket_text','Category','Tags']].apply(lambda x: response_3(prompt_3, x[0],x[1],x[2]),axis=1)
end = time.time()

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2509.16 ms
llama_print_timings:      sample time =       7.27 ms /    15 runs   (    0.48 ms per token,  2062.14 tokens per second)
llama_print_timings: prompt eval time =    3136.58 ms /   313 tokens (   10.02 ms per token,    99.79 tokens per second)
llama_print_timings:        eval time =   11691.85 ms /    14 runs   (  835.13 ms per token,     1.20 tokens per second)
llama_print_timings:       total time =   14888.88 ms /   327 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    2509.16 ms
llama_print_timings:      sample time =       7.69 ms /    15 runs   (    0.51 ms per token,  1949.57 tokens per second)
llama_print_timings: prompt eval time =    1904.64 ms /    82 tokens (   23.23 ms per token,    43.05 tokens per second)
llama_print_timings:        eval time =   11693.34 ms /    14 runs   (  835.24 ms per token,     1.20 tokens per second)
llama_print_timings:       to

In [ ]:
print("Time taken ",(end-start))

Time taken  317.72684264183044


In [ ]:
data_3['model_response']

0                   {"Priority": "High", "ETA": "ASAP"}
1                   {"Priority": "High", "ETA": "ASAP"}
2                   {"Priority": "High", "ETA": "ASAP"}
3      {"Priority": "Medium", "ETA": "Within 24 hours"}
4      {"Priority": "Medium", "ETA": "Within 24 hours"}
5                   {"Priority": "High", "ETA": "ASAP"}
6        {"Priority": "High", "ETA": "Within 24 hours"}
7                   {"Priority": "High", "ETA": "ASAP"}
8                   {"Priority": "High", "ETA": "ASAP"}
9        {"Priority": "High", "ETA": "Within 24 hours"}
10                  {"Priority": "High", "ETA": "ASAP"}
11                  {"Priority": "High", "ETA": "ASAP"}
12                  {"Priority": "High", "ETA": "ASAP"}
13                  {"Priority": "High", "ETA": "ASAP"}
14     {"Priority": "Medium", "ETA": "Within 48 hours"}
15                  {"Priority": "High", "ETA": "ASAP"}
16    {"Priority": "Medium", "ETA": "Within 2-4 hours"}
17                  {"Priority": "High", "ETA": 

In [ ]:
i = 2
print(data_3.loc[i, 'support_ticket_text'])

I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?


In [ ]:
print(data_3.loc[i, 'model_response'])

{"Priority": "High", "ETA": "ASAP"}


In [ ]:
# Applying the function to the model response
data_3['model_response_parsed'] = data_3['model_response'].apply(extract_json_data)
data_3['model_response_parsed'].head()

0                 {'Priority': 'High', 'ETA': 'ASAP'}
1                 {'Priority': 'High', 'ETA': 'ASAP'}
2                 {'Priority': 'High', 'ETA': 'ASAP'}
3    {'Priority': 'Medium', 'ETA': 'Within 24 hours'}
4    {'Priority': 'Medium', 'ETA': 'Within 24 hours'}
Name: model_response_parsed, dtype: object

In [ ]:
# Normalizing the model_response_parsed column
model_response_parsed_df_3 = pd.json_normalize(data_3['model_response_parsed'])
model_response_parsed_df_3.head(21)

,Priority,ETA
0,High,ASAP
1,High,ASAP
2,High,ASAP
3,Medium,Within 24 hours
4,Medium,Within 24 hours
5,High,ASAP
6,High,Within 24 hours
7,High,ASAP
8,High,ASAP
9,High,Within 24 hours


In [ ]:
# Concatinating two dataframes
data_with_parsed_model_output_3 = pd.concat([data_3, model_response_parsed_df_3], axis=1)
data_with_parsed_model_output_3.head()

,support_tick_id,support_ticket_text,model_response,model_response_parsed,Priority,ETA
0,ST2023-006,My internet connection has significantly slowe...,"{""Priority"": ""High"", ""ETA"": ""ASAP""}","{'Priority': 'High', 'ETA': 'ASAP'}",High,ASAP
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{""Priority"": ""High"", ""ETA"": ""ASAP""}","{'Priority': 'High', 'ETA': 'ASAP'}",High,ASAP
2,ST2023-008,I've accidentally deleted essential work docum...,"{""Priority"": ""High"", ""ETA"": ""ASAP""}","{'Priority': 'High', 'ETA': 'ASAP'}",High,ASAP
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{""Priority"": ""Medium"", ""ETA"": ""Within 24 hours""}","{'Priority': 'Medium', 'ETA': 'Within 24 hours'}",Medium,Within 24 hours
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","{""Priority"": ""Medium"", ""ETA"": ""Within 24 hours""}","{'Priority': 'Medium', 'ETA': 'Within 24 hours'}",Medium,Within 24 hours


In [ ]:
# Dropping model_response and model_response_parsed columns
final_data_3 = data_with_parsed_model_output_3.drop(['model_response','model_response_parsed'], axis=1)
final_data_3.head()

,support_tick_id,support_ticket_text,Priority,ETA
0,ST2023-006,My internet connection has significantly slowe...,High,ASAP
1,ST2023-007,Urgent help required! My laptop refuses to sta...,High,ASAP
2,ST2023-008,I've accidentally deleted essential work docum...,High,ASAP
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Medium,Within 24 hours
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Medium,Within 24 hours


In [ ]:
final_data_3 = pd.concat([final_data_3,final_data_2[["Category","Tags"]]],axis=1)

In [ ]:
final_data_3 = final_data_3[["support_tick_id","support_ticket_text","Category","Tags","Priority","ETA"]]

In [ ]:
final_data_3

,support_tick_id,support_ticket_text,Category,Tags,Priority,ETA
0,ST2023-006,My internet connection has significantly slowe...,Technical issues,"[Internet connection, Slow internet, Disconnec...",High,ASAP
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware issues,"[Hardware issue, Laptop start-up, Critical pre...",High,ASAP
2,ST2023-008,I've accidentally deleted essential work docum...,Data recovery,"[data loss, data recovery]",High,ASAP
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Technical issues,"[Wi-Fi, Signal, Weak, Troubleshooting]",Medium,Within 24 hours
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Technical issues,"[Battery, Draining, Smartphone]",Medium,Within 24 hours
5,ST2023-011,I'm locked out of my online banking account an...,Technical issues,"[Password reset, Online banking, Account access]",High,ASAP
6,ST2023-012,"My computer's performance is sluggish, severel...",Technical issues,"[performance, optimization]",High,Within 24 hours
7,ST2023-013,I'm experiencing a recurring blue screen error...,Hardware issues,"[Blue Screen, Hardware, Crash]",High,ASAP
8,ST2023-014,My external hard drive isn't being recognized ...,Data recovery,"[External Hard Drive, Data Recovery]",High,ASAP
9,ST2023-015,The graphics card in my gaming laptop seems to...,Hardware issues,"[Graphics Card, Hardware, Performance]",High,Within 24 hours


## Task 4 - Creating a Draft Response

In [ ]:
# creating a copy of the data
data_4 = data.copy()

In [ ]:
def response_4(prompt,ticket,category,tags,priority,eta):
    start = time.time()
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      Category : {category}
      Tags : {tags}
      Priority: {priority}
      ETA: {eta}
      A:
      """,
      max_tokens=80,
      stop=["Q:", "\n"],
      temperature=0.2,
      echo=False,
    )
    end = time.time()

    temp_output = model_output["choices"][0]["text"]


    return temp_output

In [ ]:
prompt_4 = """
    You are a technical assistant. You are provided a support ticket, category of the ticket,tags,the priority with which it has to be resolved, and the ETA for resolution.
    Based on this information, draft a response that can be shared with the customer.

    Instructions:
    1. Carefully read the text of the support ticket and consider the overall sentiment of the review
    2. Use the tags and category for summarizing.
    3. The ETA should be mentioned in the response to the customer
    4. The response should be empathetic in nature
    5. The number of words in the response should be less than 48.

    Return only the response that has to be shared with the customer, no additional information should be present in the output.
"""

In [ ]:
#Applying generate_llama_response function on support_ticket_text column
start = time.time()
data_4['model_response'] = final_data_3[['support_ticket_text','Category','Tags','Priority','ETA']].apply(lambda x: response_4(prompt_4, x[0],x[1],x[2],x[3],x[4]),axis=1)
end = time.time()

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2509.16 ms
llama_print_timings:      sample time =      44.28 ms /    80 runs   (    0.55 ms per token,  1806.56 tokens per second)
llama_print_timings: prompt eval time =    2662.45 ms /   253 tokens (   10.52 ms per token,    95.03 tokens per second)
llama_print_timings:        eval time =   65648.28 ms /    79 runs   (  830.99 ms per token,     1.20 tokens per second)
llama_print_timings:       total time =   68646.48 ms /   332 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    2509.16 ms
llama_print_timings:      sample time =       3.38 ms /     6 runs   (    0.56 ms per token,  1773.05 tokens per second)
llama_print_timings: prompt eval time =    2240.76 ms /    95 tokens (   23.59 ms per token,    42.40 tokens per second)
llama_print_timings:        eval time =    4082.04 ms /     5 runs   (  816.41 ms per token,     1.22 tokens per second)
llama_print_timings:       to

In [ ]:
print("Time taken",(end-start))

Time taken 699.6000006198883


In [ ]:
data_4['model_response'].values

array(["We're truly sorry for the inconvenience you're experiencing with your internet connection. Our team is currently working to identify and resolve the issue tagged as 'Slow internet' and 'Disconnections' in your account (Category: Technical issues). We understand the importance of a stable connection, especially while working from home. Our top priority is to get your connection back up to speed",
       ' Dear Valued Customer,',
       ' We deeply apologize for your data loss issue (#data loss). Our team will prioritize your data recovery request (#data recovery) with high priority (Priority: High) and work towards an ASAP resolution (ETA: ASAP). We understand the urgency of your situation and will keep you updated throughout the process.',
       ' Dear Valued Customer,', ' Dear Valued Customer,',
       " We're truly sorry for the inconvenience you're experiencing with your online banking account access. Our team is working diligently to help you reset your password as soon as

In [ ]:
i = 2
print(data_4.loc[i, 'support_ticket_text'])

I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?


In [ ]:
print(data_4.loc[i, 'model_response'])

 We deeply apologize for your data loss issue (#data loss). Our team will prioritize your data recovery request (#data recovery) with high priority (Priority: High) and work towards an ASAP resolution (ETA: ASAP). We understand the urgency of your situation and will keep you updated throughout the process.


In [ ]:
final_data_4 = pd.concat([final_data_3,data_4["model_response"]],axis=1)

In [ ]:
final_data_4.rename(columns={"model_response":"Response"},inplace=True)

In [ ]:
final_data_4

,support_tick_id,support_ticket_text,Category,Tags,Priority,ETA,Response
0,ST2023-006,My internet connection has significantly slowe...,Technical issues,"[Internet connection, Slow internet, Disconnec...",High,ASAP,We're truly sorry for the inconvenience you're...
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware issues,"[Hardware issue, Laptop start-up, Critical pre...",High,ASAP,"Dear Valued Customer,"
2,ST2023-008,I've accidentally deleted essential work docum...,Data recovery,"[data loss, data recovery]",High,ASAP,We deeply apologize for your data loss issue ...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Technical issues,"[Wi-Fi, Signal, Weak, Troubleshooting]",Medium,Within 24 hours,"Dear Valued Customer,"
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Technical issues,"[Battery, Draining, Smartphone]",Medium,Within 24 hours,"Dear Valued Customer,"
5,ST2023-011,I'm locked out of my online banking account an...,Technical issues,"[Password reset, Online banking, Account access]",High,ASAP,We're truly sorry for the inconvenience you'r...
6,ST2023-012,"My computer's performance is sluggish, severel...",Technical issues,"[performance, optimization]",High,Within 24 hours,We're truly sorry for the inconvenience you'r...
7,ST2023-013,I'm experiencing a recurring blue screen error...,Hardware issues,"[Blue Screen, Hardware, Crash]",High,ASAP,"Dear Valued Customer,"
8,ST2023-014,My external hard drive isn't being recognized ...,Data recovery,"[External Hard Drive, Data Recovery]",High,ASAP,We're truly sorry for the inconvenience you'r...
9,ST2023-015,The graphics card in my gaming laptop seems to...,Hardware issues,"[Graphics Card, Hardware, Performance]",High,Within 24 hours,"Dear Valued Customer,"


## Model Output Analysis

In [ ]:
# Creating a copy of the dataframe of task-4
final_data = final_data_4.copy()

In [ ]:
final_data['Category'].value_counts()    # complete the code with the column name for the column containing ticket categories

Technical issues    9
Data recovery       7
Hardware issues     5
Name: Category, dtype: int64

* The most common support category is "Technical Issues" with 9 occurrences, followed by "Data Recovery" with 7 occurrences, and "Hardware Issues" is the third most common category with 5 occurrences.

In [ ]:
final_data["Priority"].value_counts()

High      17
Medium     4
Name: Priority, dtype: int64

* The model identified 17 tickets to be of high priority and 4 to be of medium priority.
* No tickets were assigned as low priority.

In [ ]:
final_data["ETA"].value_counts()

ASAP                14
Within 24 hours      4
Within 48 hours      2
Within 2-4 hours     1
Name: ETA, dtype: int64

* The presence of an "ASAP" ETA for tickets suggests the existence of critical issues that require immediate attention.

Let's dive in a bit deeper here.

In [ ]:
final_data.groupby(['Category', 'ETA']).support_tick_id.count()

Category          ETA             
Data recovery     ASAP                7
Hardware issues   ASAP                3
                  Within 24 hours     1
                  Within 48 hours     1
Technical issues  ASAP                4
                  Within 2-4 hours    1
                  Within 24 hours     3
                  Within 48 hours     1
Name: support_tick_id, dtype: int64

- Data Recovery:

   - All the tickets in the "Data Recovery" category have an ETA of "ASAP".

- Hardware Issues:

   - The majority of tickets in the "Hardware Issues" category have an ETA of "ASAP"
   - There are also tickets with ETAs of "Within 24 hours" and "Within 48 hours"

- Technical Issues:

   - Tickets in the "Technical Issues" category have varied ETAs.
   - The most common ETA is "ASAP".

- ETA Distribution:

   - "ASAP" is a common ETA across different categories.
   - There's a mix of shorter and longer ETAs, indicating varying levels of urgency for issue resolution.

- Priority

    - There are no support tickets with low priority, which is unlikely in the real world.